In [147]:
import pandas as pd
import numpy as np

# Load Results

In [148]:
RESULTS_FILE = './results.csv'
NUMBER_OF_EXPERIMENTS = 27 * 228
PARAM_COLUMNS = ['fitness_function', 'use_constraint', 'constraint_handling_method', 'constraint',
                 'particle_speed_limit', 'population_size', 'personal_confidence', 'swarm_confidence',
                 'particle_inertia', 'constraint_r']
RESULT_COLUMNS_ALL = ['fitness', 'optimum', 'iterations', 'optimum_reached', 'iterations_to_opt']#
RESULT_COLUMNS = ['fitness', 'optimum', 'optimum_reached', 'iterations_to_opt']

In [149]:
df_all_iterations = pd.read_csv(RESULTS_FILE)
df_all_iterations.head()

,number,iteration,fitness_function,use_constraint,constraint_handling_method,constraint,particle_speed_limit,population_size,personal_confidence,swarm_confidence,particle_inertia,constraint_r,fitness,optimum,iterations,optimum_reached,iterations_to_opt
0,10001,1,Shubert function,False,NaN,NaN,13,5,0.5,1.5,0.1,-0.15,0.749117,1.0,500,False,9
1,10002,1,Shubert function,False,NaN,NaN,13,5,1.0,1.0,0.1,-0.15,0.641035,1.0,500,False,33
2,10003,1,Shubert function,False,NaN,NaN,13,5,1.5,0.5,0.1,-0.15,0.698651,1.0,500,False,0
3,10004,1,Shubert function,False,NaN,NaN,13,5,0.5,1.5,0.5,-0.15,0.749117,1.0,500,False,15
4,10005,1,Shubert function,False,NaN,NaN,13,5,1.0,1.0,0.5,-0.15,0.749117,1.0,500,False,15


# Experiment Parameters

In [150]:
df_params = df_all_iterations[df_all_iterations.iteration == 1].set_index('number')[PARAM_COLUMNS]
df_params.constraint_r.replace(-0.15, np.nan, inplace=True)  # default was used when no constraint used -> change to NAN
assert df_params.index.is_unique
assert df_params.shape == (NUMBER_OF_EXPERIMENTS, len(PARAM_COLUMNS))
df_params.head()

,fitness_function,use_constraint,constraint_handling_method,constraint,particle_speed_limit,population_size,personal_confidence,swarm_confidence,particle_inertia,constraint_r
number,,,,,,,,,,
10001,Shubert function,False,NaN,NaN,13,5,0.5,1.5,0.1,NaN
10002,Shubert function,False,NaN,NaN,13,5,1.0,1.0,0.1,NaN
10003,Shubert function,False,NaN,NaN,13,5,1.5,0.5,0.1,NaN
10004,Shubert function,False,NaN,NaN,13,5,0.5,1.5,0.5,NaN
10005,Shubert function,False,NaN,NaN,13,5,1.0,1.0,0.5,NaN


In [151]:
# sanity check params
assert set(df_params.fitness_function.unique()) == set(['Shubert function', "Booth's function", 'Schwefel function'])#
assert set(df_params.use_constraint.unique()) == set([True, False])
assert set(df_params.constraint_handling_method.unique()) == set([np.nan, 'Penalty Method', 'Rejection Method'])
assert set(df_params.constraint.unique()) == set([np.nan, 'Constraint 3', 'Constraint 5', 'Constraint 10'])
# assert set(df_params.particle_speed_limit.unique()) == set([2, 10, 19])
assert df_params.particle_speed_limit.unique() == [13] # redo some experiments with variation
assert set(df_params.population_size.unique()) == set([5, 10, 35, 80])
assert set(df_params.personal_confidence.unique()) == set([0.5, 1.0, 1.5])
assert set(df_params.swarm_confidence.unique()) == set([0.5, 1.0, 1.5])
assert set(df_params.particle_inertia.unique()) == set([0.1, 0.5, 0.9])
assert set(df_params.constraint_r.dropna().unique()) == set([-1.0, -1.5, -2.0])
assert df_params[~df_params.use_constraint][['constraint_handling_method', 'constraint', 'constraint_r']].isna().all(axis=None)
assert df_params[df_params.use_constraint][['constraint_handling_method', 'constraint', 'constraint_r']].notna().all(axis=None)

In [152]:
# inspect values in Experiment setup
for column in PARAM_COLUMNS:
    print(f'{column}: {df_params[column].unique()}')

fitness_function: ['Shubert function' "Booth's function" 'Schwefel function']
use_constraint: [False  True]
constraint_handling_method: [nan 'Penalty Method' 'Rejection Method']
constraint: [nan 'Constraint 3' 'Constraint 5' 'Constraint 10']
particle_speed_limit: [13]
population_size: [ 5 10 35 80]
personal_confidence: [0.5 1.  1.5]
swarm_confidence: [1.5 1.  0.5]
particle_inertia: [0.1 0.5 0.9]
constraint_r: [ nan -1.  -1.5 -2. ]


# Experiment Results

In [153]:
df_results = df_all_iterations[['number'] + RESULT_COLUMNS_ALL]
df_results = df_results.groupby('number').mean()
## sanity check results
assert df_results.shape == (NUMBER_OF_EXPERIMENTS, len(RESULT_COLUMNS_ALL))
assert df_results.fitness.min() >= 0.0
assert df_results.fitness.max() <= 1.0
assert df_results.optimum.min() > 0.0
assert df_results.optimum.max() <= 1.0
assert df_results.iterations.unique() == [500.0]
assert df_results.optimum_reached.min() >= 0.0
assert df_results.optimum_reached.max() <= 1.0
assert df_results.iterations_to_opt.min() >= 0
assert df_results.iterations_to_opt.max() <= 500
df_results.head()

,fitness,optimum,iterations,optimum_reached,iterations_to_opt
number,,,,,
10001,0.685586,1.0,500.0,0.0,8.5
10002,0.632124,1.0,500.0,0.0,144.0
10003,0.700413,1.0,500.0,0.0,5.0
10004,0.686165,1.0,500.0,0.0,14.0
10005,0.686165,1.0,500.0,0.0,21.5


In [154]:
df_results[RESULT_COLUMNS].describe()

,fitness,optimum,optimum_reached,iterations_to_opt
count,6156.000000,6156.000000,6156.000000,6156.000000
mean,0.896854,0.930320,0.550520,39.536712
std,0.158699,0.152377,0.466556,65.502258
min,0.471753,0.495829,0.000000,0.000000
25%,0.874587,0.924913,0.000000,8.000000
50%,0.984247,1.000000,0.500000,16.000000
75%,1.000000,1.000000,1.000000,38.500000
max,1.000000,1.000000,1.000000,498.000000


# Combined Data by Experiment

In [155]:
df = df_params.join(df_results)
assert df.shape == (NUMBER_OF_EXPERIMENTS, len(PARAM_COLUMNS) + len(RESULT_COLUMNS_ALL))
df.head()

,fitness_function,use_constraint,constraint_handling_method,constraint,particle_speed_limit,population_size,personal_confidence,swarm_confidence,particle_inertia,constraint_r,fitness,optimum,iterations,optimum_reached,iterations_to_opt
number,,,,,,,,,,,,,,,
10001,Shubert function,False,NaN,NaN,13,5,0.5,1.5,0.1,NaN,0.685586,1.0,500.0,0.0,8.5
10002,Shubert function,False,NaN,NaN,13,5,1.0,1.0,0.1,NaN,0.632124,1.0,500.0,0.0,144.0
10003,Shubert function,False,NaN,NaN,13,5,1.5,0.5,0.1,NaN,0.700413,1.0,500.0,0.0,5.0
10004,Shubert function,False,NaN,NaN,13,5,0.5,1.5,0.5,NaN,0.686165,1.0,500.0,0.0,14.0
10005,Shubert function,False,NaN,NaN,13,5,1.0,1.0,0.5,NaN,0.686165,1.0,500.0,0.0,21.5


# Evaluation

In [158]:
df[~df.use_constraint].groupby(['fitness_function', 'personal_confidence', 'swarm_confidence'])[RESULT_COLUMNS].mean()

fitness  optimum  \
fitness_function  personal_confidence swarm_confidence                      
Booth's function  0.5                 1.5               0.998885      1.0   
                  1.0                 1.0               0.999087      1.0   
                  1.5                 0.5               0.999366      1.0   
Schwefel function 0.5                 1.5               0.954016      1.0   
                  1.0                 1.0               0.930546      1.0   
                  1.5                 0.5               0.912609      1.0   
Shubert function  0.5                 1.5               0.842697      1.0   
                  1.0                 1.0               0.870071      1.0   
                  1.5                 0.5               0.902260      1.0   

                                                        optimum_reached  \
fitness_function  personal_confidence swarm_confidence                    
Booth's function  0.5                 1.5                      0.833333   
                  1.0                 1.0                      0.833333   
                  1.5                 0.5                      0.791667   
Schwefel function 0.5                 1.5                      0.625000   
                  1.0                 1.0                      0.583333   
                  1.5                 0.5                      0.416667   
Shubert function  0.5                 1.5                      0.500000   
                  1.0                 1.0                      0.625000   
                  1.5                 0.5                      0.500000   

                                                        iterations_to_opt  
fitness_function  personal_confidence swarm_confidence                     
Booth's function  0.5                 1.5                       13.791667  
                  1.0                 1.0                       14.166667  
                  1.5                 0.5                       29.375000  
Schwefel function 0.5                 1.5                       84.250000  
                  1.0                 1.0                       80.916667  
                  1.5                 0.5                       69.916667  
Shubert function  0.5                 1.5                       15.166667  
                  1.0                 1.0                       26.750000  
                  1.5                 0.5                      111.000000